# Multithreading Support

Before starting Jupyter server we need to set `JULIA_NUM_THREADS` and `PYTHON_JULIACALL_HANDLE_SIGNALS` environment variables. This enables us to start Julia with multiple threads and use experimental multithreading support in JuliaCall:
```bash
export JULIA_NUM_THREADS=4
export PYTHON_JULIACALL_HANDLE_SIGNALS=yes
```

In [ ]:
from juliacall import Main as jl

In [ ]:
%load_ext juliacall

In [ ]:
%%julia

using Base.Threads
println(Threads.nthreads())  # This will print the number of threads

In [ ]:
%%julia

using Pkg
Pkg.add("UnROOT")
using UnROOT
using AwkwardArray

In [ ]:
tree = jl.Main.ROOTFile("./data/SMHiggsToZZTo4L.root")

In [ ]:
%%julia

tree = ROOTFile("./data/SMHiggsToZZTo4L.root")
events = LazyTree(tree, "Events")

In [ ]:
%%julia

using LorentzVectorHEP
using Base.Threads

function main_looper_over_vector(events)
    # Pre-allocate an AwkwardArray to store Higgs mass values
    array = Vector{Float64}(undef, length(events))  # Standard vector for simpler threading
    count = Threads.Atomic{Int}(1)  # Use atomic integer to track valid entries across threads

    @threads for i in 1:length(events)
        evt = events[i]
        
        # Destructure the necessary fields from the event
        (; Muon_charge::Vector{Float64}, Muon_pt::Vector{Float64}, Muon_eta::Vector{Float64}, 
            Muon_phi::Vector{Float64}, Muon_mass::Vector{Float64}) = evt

        # Skip events that don't meet the required conditions
        if length(Muon_charge) != 4 || sum(Muon_charge) != 0
            continue
        end

        # Compute the Lorentz vector sum and Higgs mass
        higgs_4vector = sum(LorentzVectorCyl.(Muon_pt, Muon_eta, Muon_phi, Muon_mass))
        higgs_mass = mass(higgs_4vector)

        # Use atomic operations to safely update the shared array
        idx = Threads.atomic_add!(count, 1)
        array[idx] = higgs_mass
    end

    # Return the portion of the array that contains valid results
    return array[1:count[]-1]
end

In [ ]:
%%julia

array = @time main_looper_over_vector(events)

In [ ]:
%%julia

using LorentzVectorHEP
using Base.Threads

function faster_main_looper_over_vector(events)
    # Pre-allocate a results array with a large enough size for simpler threading
    n_events = length(events)
    max_results = div(n_events, Threads.nthreads()) + 1
    thread_results = [Vector{Float64}(undef, max_results) for _ in 1:Threads.nthreads()]

    # Thread-local counters
    thread_counts = fill(1, Threads.nthreads())

    @threads for i in 1:n_events
        evt = events[i]

        # Destructure the necessary fields from the event
        (; Muon_charge::Vector{Float64}, Muon_pt::Vector{Float64}, Muon_eta::Vector{Float64}, 
            Muon_phi::Vector{Float64}, Muon_mass::Vector{Float64}) = evt

        # Skip invalid events
        if length(Muon_charge) != 4 || sum(Muon_charge) != 0
            continue
        end

        # Compute the Lorentz vector sum and Higgs mass
        higgs_4vector = sum(LorentzVectorCyl.(Muon_pt, Muon_eta, Muon_phi, Muon_mass))
        higgs_mass = mass(higgs_4vector)

        # Each thread writes to its own result array
        tid = threadid()
        thread_results[tid][thread_counts[tid]] = higgs_mass
        thread_counts[tid] += 1
    end

    # Combine results from all threads into a single array
    final_result = vcat([thread_results[tid][1:thread_counts[tid]-1] for tid in 1:Threads.nthreads()]...)

    return final_result
end

In [ ]:
%%julia

array = @time faster_main_looper_over_vector(events)

In [ ]:
%%julia

using LorentzVectorHEP
using Base.Threads

function even_faster_main_looper_over_vector(events)
    # Pre-allocate a results array with a large enough size for simpler threading
    n_events = length(events)
    max_results = div(n_events, Threads.nthreads()) + 1
    thread_results = [Vector{Float64}(undef, max_results) for _ in 1:Threads.nthreads()]

    # Thread-local counters
    thread_counts = fill(1, Threads.nthreads())

    @threads for i in 1:n_events
        evt = events[i]

        # Destructure the necessary fields from the event
        (; Muon_charge::Vector{Float64}, Muon_pt::Vector{Float64}, Muon_eta::Vector{Float64}, 
            Muon_phi::Vector{Float64}, Muon_mass::Vector{Float64}) = evt
        # (; Muon_charge, Muon_pt, Muon_eta, Muon_phi, Muon_mass) = evt

        # Skip invalid events
        if length(Muon_charge) != 4 || sum(Muon_charge) != 0
            continue
        end

        # Compute the Lorentz vector sum and Higgs mass
        higgs_4vector = sum(LorentzVectorCyl.(Muon_pt, Muon_eta, Muon_phi, Muon_mass))
        higgs_mass = mass(higgs_4vector)

        # Each thread writes to its own result array
        tid = threadid()
        thread_results[tid][thread_counts[tid]] = higgs_mass
        thread_counts[tid] += 1
    end

    # Combine results from all threads into a single array
    final_result = vcat([thread_results[tid][1:thread_counts[tid]-1] for tid in 1:Threads.nthreads()]...)

    return final_result
end

In [ ]:
%%julia

array = @time even_faster_main_looper_over_vector(events)

In [ ]:
%%julia

array = @time main_looper_over_vector(events)

In [ ]:
%%julia

typeof(events)

In [ ]:
%%julia

using AwkwardArray
using Base.Threads

function main_looper_awkward(events)
    array = AwkwardArray.PrimitiveArray{Float64}()
    lock_obj = ReentrantLock()  # Create a lock object to control access to shared array

    @threads for i in 1:length(events)
        evt = events[i]

        # Destructure the necessary fields from the event
        (; Muon_charge, Muon_pt, Muon_eta, Muon_phi, Muon_mass) = evt

        # Skip event if it doesn't meet the required conditions
        if length(Muon_charge) != 4 || sum(Muon_charge) != 0
            continue
        end

        # Create Lorentz vectors for the muons and calculate the Higgs mass
        higgs_4vector = sum(LorentzVectorCyl.(Muon_pt, Muon_eta, Muon_phi, Muon_mass))
        higgs_mass = mass(higgs_4vector)

        # Use lock to safely push! into the shared array
        lock(lock_obj)  # Explicitly lock before modifying shared data
        try
            push!(array, higgs_mass)
        finally
            unlock(lock_obj)  # Ensure the lock is always released
        end
    end

    return array
end

In [ ]:
%%julia

array = @time main_looper_awkward(events)

In [ ]:
%%julia

using LorentzVectorHEP
using AwkwardArray
using Base.Threads

function main_looper(events)
    # Create an empty AwkwardArray for storing the Higgs mass values
    array = AwkwardArray.PrimitiveArray{Float64}()

    # Loop over events and process only valid ones
    @threads for i in 1:length(events)
        evt = events[i]

        # Destructure the necessary fields from the event
        (; Muon_charge, Muon_pt, Muon_eta, Muon_phi, Muon_mass) = evt

        # Skip event if it doesn't meet the required conditions
        if length(Muon_charge) != 4 || sum(Muon_charge) != 0
            continue
        end

        # Create Lorentz vectors for the muons and calculate the Higgs mass
        higgs_4vector = sum(LorentzVectorCyl.(Muon_pt, Muon_eta, Muon_phi, Muon_mass))
        higgs_mass = mass(higgs_4vector)

        # Add the result to the AwkwardArray
        push!(array, higgs_mass)
    end

    # Return the final AwkwardArray containing Higgs masses
    return array
end

In [ ]:
%%julia

array = @time main_looper(events)